# Employee Sales Performance

1. Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.
2. Write a short statement describing your results, and providing a possible interpretation.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Finding relevant data

Our main metric will be  total dollar amount of purchases assigned to a sales support agent. That means:
1. will need to query:
  * `invoice` to get total amount purchased
  * `customers` to join data about total amount purchased to employees data
  * `employees` 
2. in the `employees` table we can filter to only include employees with `Sales Support Agent` title.

To see what other attributes might influence sales performance let's explore the `employees` table first. 

In [2]:
%%sql

SELECT name FROM PRAGMA_TABLE_INFO('employee');

 * sqlite:///chinook.db
Done.


name
employee_id
last_name
first_name
title
reports_to
birthdate
hire_date
address
city
state


### Place of living

One thing that could influence sales performance is the country (some countries have more people than others and thus more purchases would be expected), but it turns out all agents so far are from Canada.

In [7]:
%%sql

SELECT DISTINCT country 
FROM employee
WHERE title = 'Sales Support Agent'
;

 * sqlite:///chinook.db
Done.


country
Canada


The same goes for city

In [8]:
%%sql

SELECT DISTINCT city 
FROM employee
WHERE title = 'Sales Support Agent'
;

 * sqlite:///chinook.db
Done.


city
Calgary


### Time working

Another obvious reason a sales manager would have more sales is that he was working for a longer period of time. We could check it by exploring the `hire_date` column.

In [9]:
%%sql
SELECT 
	first_name || ' ' || last_name AS name,
	hire_date 
FROM employee e 
WHERE title = 'Sales Support Agent'
;

 * sqlite:///chinook.db
Done.


name,hire_date
Jane Peacock,2017-04-01 00:00:00
Margaret Park,2017-05-03 00:00:00
Steve Johnson,2017-10-17 00:00:00


The agents indeed were not hired simultaniously, so we need to take it into account and probably group data by month.

### Other relevant data

There might be some other reasons why any particular sales agent doing better or worse:
* he might be just lucky to first serve a very large client (something like Apple)
* there might be some promotion going on that results in more sales, while one of the Agents is on the vacation

And there might be others. So there needs to be an understanding that these metrics might be not very indicative especially given there is so little data - only three sales agents and we don't have any data at all to check against those extra scenarios.

### Chosen source data

So we end up with 3 time series (say per day or month) for each sales agent measuring how much dollars in total they've made.

In [10]:
%%sql

SELECT 
	e.first_name || ' ' || e.last_name AS name,
	i.invoice_date,
	sum(i.total) AS total
FROM invoice i 
INNER JOIN customer c
	ON c.customer_id = i.customer_id 
INNER JOIN employee e
	ON e.employee_id = c.support_rep_id 
GROUP BY 
	1,
	2 
;

 * sqlite:///chinook.db
Done.


name,invoice_date,total
Jane Peacock,2017-01-03 00:00:00,25.740000000000002
Jane Peacock,2017-01-06 00:00:00,7.92
Jane Peacock,2017-01-13 00:00:00,9.9
Jane Peacock,2017-01-18 00:00:00,8.91
Jane Peacock,2017-01-20 00:00:00,10.89
Jane Peacock,2017-01-21 00:00:00,3.96
Jane Peacock,2017-01-25 00:00:00,3.96
Jane Peacock,2017-01-26 00:00:00,8.91
Jane Peacock,2017-02-02 00:00:00,4.95
Jane Peacock,2017-02-06 00:00:00,9.9


Ok, this turned out to be not very human readable and we could do visualization of these time series (or grouping say by month) to see how well is every sales agent is doing comparatively to each other and what are the tendancies, but want we want now is simplier - we just need to see how much overall every agent has sold. We can skip the 2017 year just to make comparison fair for those hired later.

In [12]:
%%sql
SELECT 
	e.first_name || ' ' || e.last_name AS name,
	sum(i.total) AS total
FROM invoice i 
INNER JOIN customer c
	ON c.customer_id = i.customer_id 
INNER JOIN employee e
	ON e.employee_id = c.support_rep_id 
WHERE i.invoice_date >= '2018-01-01 00:00:00'
GROUP BY 
	1
ORDER BY 2 DESC
;

 * sqlite:///chinook.db
Done.


name,total
Jane Peacock,1198.8900000000012
Margaret Park,1167.2100000000007
Steve Johnson,1141.4700000000007


## Conclusion

We can see that agents are very close in their results with Jane Peacock leading